SIGNATURE VERIFICATION 

IMPORT THE LIBRARIES

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from glob import glob



READING THE DATA PATH (SigComp Dataset)

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = "/content/drive/MyDrive/sign_data_mod_mod/train"
test_path  = "/content/drive/MyDrive/sign_data_mod_mod/test"

DATA AUGMENTATION

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

TRAINING DATASET

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

TESTING DATASET

In [ ]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
training_set.class_indices

In [ ]:
test_set.class_indices


DATA AUGMENTED IMAGES




In [ ]:
def Show_Image(img_arr):
  fig,axes=plt.subplots(1,5,figsize=(20,20))
  axes=axes.flatten()

  for img,ax in zip(img_arr,axes):
    ax.imshow(img)
  
  plt.tight_layout()
  plt.show()

In [ ]:
images=[training_set[0][0][0] for i in range(6)]
Show_Image(images)

In [ ]:
images=[test_set[0][0][0] for i in range(6)]
Show_Image(images)

MOBILENETV2 MODEL FOR TRAINING

In [ ]:
model_mobilenet=MobileNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in model_mobilenet.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/sign_data_mod_mod/train/*')

In [ ]:
folders

In [ ]:
x = Flatten()(model_mobilenet.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)


In [ ]:
model = Model(model_mobilenet.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=[tensorflow.keras.metrics.AUC(),"accuracy"])


MODEL TRAINING

In [ ]:
result = model.fit(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

LOSS AND ACCURACY

In [ ]:
plt.plot(result.history['loss'], label='Training Loss')
plt.plot(result.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()
plt.savefig('Loss_Value')


plt.plot(result.history['accuracy'], label='Training Accuracy')
plt.plot(result.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()
plt.savefig('Accuracy_Value')

THE FINAL MODEL

In [ ]:
model.save('/content/drive/MyDrive/sign_data_mod_mod/Model_MobileNetV2.h5')